In [ ]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


In [1]:
import numpy as np
import random as rand
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
# Função objetivo
def f6(x, y):
    return 0.5-((np.sin(np.sqrt(x**2+y**2))**2)-0.5)/(1+0.001*(x**2+y**2))**2

def bin2real(bin_string):
    inteiro=int(bin_string, 2)          # bin2int
    #(inteiro*(100-(-100))/2**25)-100
    return (inteiro*200/33554431)-100   # int2real

def real2bin(real):
    inteiro = int((real+100)*33554431/200)  # real2int
    return bin(inteiro)[2:].zfill(25)       # int2bin

def particao(z):
    soma_total= sum(z)
    return [aptidao/soma_total for aptidao in z]

def gerar_pop(Populacao):
    xbin = [''.join(rand.choices(['0', '1'], k=25)) for _ in range(Populacao)]
    ybin = [''.join(rand.choices(['0', '1'], k=25)) for _ in range(Populacao)]
    xreal = [bin2real(individuo) for individuo in xbin]
    yreal = [bin2real(individuo) for individuo in ybin]
    z = [f6(x, y) for x, y in zip(xreal, yreal)]
    tam_particao = particao(z)
    return pd.DataFrame({
        'xbin': xbin,
        'ybin': ybin,
        'xreal': xreal,
        'yreal': yreal,
        'z': z,
        'particao': tam_particao
        })

def roleta(df_particao):    # Seleção por roleta proporcional à aptidão
    numero = rand.random()
    for iterator in range(len(df_particao)):
        numero -= df_particao.loc[iterator]
        if numero <= 0:
            return iterator
    return len(df_particao)-1

In [77]:
def mut_bin(bits, Tmut):
    bits = list(bits)
    for i in range(len(bits)):
        if rand.random() <= Tmut:
            bits[i] = '1' if bits[i] == '0' else '0'
    bits = "".join(bits)
    return bits

def cruz_bin(geracao_anterior, Tcruz, Tmut):
    xbin = []
    ybin = []
    while len(xbin) < 200: # populacao de 200 individuos
        P1 = geracao_anterior.loc[roleta(geracao_anterior['particao']), ['xbin', 'ybin']]
        P2 = geracao_anterior.loc[roleta(geracao_anterior['particao']), ['xbin', 'ybin']]
        if rand.random() > Tcruz:       # copia os genitores
            xbin.append(P1['xbin'])
            ybin.append(P1['ybin'])
            xbin.append(P2['xbin'])
            ybin.append(P2['ybin'])
        else:
            # Cruzamento por ponto de corte
            ponto_corte = rand.randint(1, 24)
            xbin.append(P1['xbin'][0:ponto_corte]+P2['xbin'][ponto_corte:25])
            xbin.append(P2['xbin'][0:ponto_corte]+P1['xbin'][ponto_corte:25])
            ponto_corte = rand.randint(1, 24)
            ybin.append(P1['ybin'][0:ponto_corte]+P2['ybin'][ponto_corte:25])
            ybin.append(P2['ybin'][0:ponto_corte]+P1['ybin'][ponto_corte:25])
    # Mutacao
    xbin = [mut_bin(x, Tmut) for x in xbin]
    ybin = [mut_bin(y, Tmut) for y in ybin]

    xreal = [bin2real(individuo) for individuo in xbin]
    yreal = [bin2real(individuo) for individuo in ybin]
    z = [f6(x, y) for x, y in zip(xreal, yreal)]
    tam_particao = particao(z)
    return pd.DataFrame({
        'xbin': xbin,
        'ybin': ybin,
        'xreal': xreal,
        'yreal': yreal,
        'z': z,
        'particao': tam_particao
        })

In [81]:
def grafico_geracao(df, geracao, pasta_save):
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(df['xreal'], df['yreal'], df['z'], c='b', marker='o')
    ax.set_xlim(-100, 100)
    ax.set_ylim(-100, 100)
    ax.set_zlim(0, 1)

    ax.set_xlabel('X real')
    ax.set_ylabel('Y real')
    ax.set_zlabel('Z (f6)')
    ax.set_title(f'Geração {geracao}')

    plt.savefig(f'{pasta_save}\\geracao{geracao}.png', dpi=300, bbox_inches='tight')
    #plt.show()
    plt.close()

def save_csv(df, geracao, pasta_save):
    df.to_csv(f'{pasta_save}\\geracao{geracao}.csv', index=False)

def plot_best(melhor_individuo, Geracoes, pasta_save):
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, Geracoes + 1), melhor_individuo)
    plt.ylim(0, 1)
    plt.xlabel('Geração')
    plt.ylabel('f6(x, y)')
    plt.title('Melhor Indivíduo por Geração')
    plt.savefig(f'{pasta_save}/melhor_individuo.png', dpi=300, bbox_inches='tight')
    #plt.show()
    plt.close()

In [ ]:
#1) tempo total: 1 min 35 segundos
Populacao = 200
Tcruz = 0.85    # ou faz ou copia os genitores
Tmut = 0.01
Geracoes = 500

melhor_individuo = []
# Primeira geracao gerada aleatoriamente
df = gerar_pop((Populacao))
grafico_geracao(df, 1, '1_images')
save_csv(df, 1, '1_csv')
melhor_individuo.append(max(df['z']))

# Numero de geracoes -1 obitidas a partir do cruzamento e mutacao da geracao anterior
for i in range(1, Geracoes):
    df = cruz_bin(df, Tcruz, Tmut)
    melhor_individuo.append(max(df['z']))
    if (i+1)%10 == 0:
        grafico_geracao(df, i+1, '1_images')
        save_csv(df, i+1, '1_csv')

plot_best(melhor_individuo, Geracoes, '1_best')


In [ ]:
def mut_real(real, Tmut, variancia):
    if rand.random() <= Tmut:
        real+=np.random.normal(loc=0, scale=np.sqrt(variancia))
        if real > 100:  # Não vai escapar dos limites
            real = 100
        if real < -100:
            real = -100
    return real

def cruz_real(geracao_anterior, Tcruz, Tmut, a, variancia):
    xreal = []
    yreal = []
    while len(xreal) < 200: # populacao de 200 individuos
        P1 = geracao_anterior.loc[roleta(geracao_anterior['particao']), ['xreal', 'yreal']]
        P2 = geracao_anterior.loc[roleta(geracao_anterior['particao']), ['xreal', 'yreal']]
        if rand.random() > Tcruz:       # copia os genitores
            xreal.append(P1['xreal'])
            yreal.append(P1['yreal'])
            xreal.append(P2['xreal'])
            yreal.append(P2['yreal'])
        else:
            # Cruzamento arimético
            xreal.append(a*P1['xreal']+(1-a)*P2['xreal'])
            xreal.append(a*P2['xreal']+(1-a)*P1['xreal'])
            yreal.append(a*P1['yreal']+(1-a)*P2['yreal'])
            yreal.append(a*P2['yreal']+(1-a)*P1['yreal'])
    # Mutacao
    xreal = [mut_real(x, Tmut, variancia) for x in xreal]
    yreal = [mut_real(y, Tmut, variancia) for y in yreal]

    xbin = [real2bin(individuo) for individuo in xreal]
    ybin = [real2bin(individuo) for individuo in yreal]
    z = [f6(x, y) for x, y in zip(xreal, yreal)]
    tam_particao = particao(z)
    return pd.DataFrame({
        'xbin': xbin,
        'ybin': ybin,
        'xreal': xreal,
        'yreal': yreal,
        'z': z,
        'particao': tam_particao
        })

In [ ]:
#2) tempo total: 1 min 38 segundos
Populacao = 200
Tcruz = 0.85    # ou faz ou copia os genitores
Tmut = 0.01
Geracoes = 500
a = 0.35
variancia = 1

melhor_individuo = []
# Primeira geracao gerada aleatoriamente
df = gerar_pop((Populacao))
grafico_geracao(df, 1, '2_images')
melhor_individuo.append(max(df['z']))

# Numero de geracoes -1 obitidas a partir do cruzamento e mutacao da geracao anterior
for i in range(1, Geracoes):
    df = cruz_real(df, Tcruz, Tmut, a, variancia)
    melhor_individuo.append(max(df['z']))
    if (i+1)%10 == 0:
        grafico_geracao(df, i+1, '2_images')

plot_best(melhor_individuo, Geracoes, '2_best')